In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import plotly.graph_objects as go
import plotly.express as px
import warnings
from tqdm import tqdm
import gc
warnings.filterwarnings("ignore")


<div class="alert alert-block alert-success" style="font-size:14px; font-family:verdana; line-height: 1.7em;">
    📌 &nbsp;<b><u>Intro:</u></b><br>
    

 En esta libreta intentamos predecir la dirección de una partícula de neutrino utilizando PCA. El Análisis de Componentes Principales (PCA) proyecta los datos desde un espacio de mayor dimensión hacia un espacio de menor dimensión al maximizar la varianza de cada componente a través de combinaciones lineales del espacio original, eliminando las correlaciones lineales y definiendo una nueva base ortogonal conocida como componentes principales. Por lo tanto, utilizaremos el primer componente principal para reconstruir la trayectoria tridimensional del neutrino encontrando la mejor combinación lineal del espacio original de características (las coordenadas x, y, z de los sensores activados).

In [2]:
from sklearn.decomposition import PCA

## <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Metricas de la competencia</p>
    


In [3]:
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Load the Data</p>

In [4]:
sensor_geometry = pd.read_csv("/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv")

In [5]:
train_meta = pq.read_pandas("/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet").to_pandas()
print(f"cantidad de eventos: {len(train_meta)}\nShape: {train_meta.shape}")

cantidad de eventos: 131953924
Shape: (131953924, 6)


In [6]:
train_meta_ = train_meta[train_meta["batch_id"] == 125]

In [7]:
del train_meta

gc.collect()

43

In [8]:
train_batch = pq.read_pandas("/kaggle/input/icecube-neutrinos-in-deep-ice/train/batch_125.parquet").to_pandas().reset_index()
print(f"cantidad de eventos: {len(train_batch)}\nShape: {train_batch.shape}")


cantidad de eventos: 34242682
Shape: (34242682, 5)


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Noisy events</p>

In [9]:
auxiliary_rate = train_batch.groupby("event_id").agg({"auxiliary" : "mean"}).reset_index().rename(columns = {"auxiliary" : "auxiliary_ratio"})


In [10]:
train_meta_ =   train_meta_.merge(auxiliary_rate, on = "event_id")
train_meta_.head(10)

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith,auxiliary_ratio
0,125,403533054,0,64,5.036863,1.047974,0.692308
1,125,403533131,65,129,5.888834,0.806877,0.523077
2,125,403533142,130,236,5.779661,1.615474,0.317757
3,125,403533173,237,286,0.193563,1.638576,0.740000
4,125,403533184,287,319,2.306770,2.450495,0.787879
5,125,403533195,320,382,5.905306,2.717603,0.682540
6,125,403533222,383,483,2.736710,1.174099,0.445545
7,125,403533241,484,632,5.194813,1.147361,0.416107
8,125,403533242,633,738,3.142933,1.921447,0.735849
9,125,403533280,739,786,3.297051,2.753681,0.625000


In [11]:
sample = train_meta_[train_meta_["auxiliary_ratio"] > 0.95].loc[54321]
train_batch_ = train_batch[train_batch["event_id"] ==sample.event_id]

In [12]:
del train_batch

gc.collect()

18

In [13]:
train_batch_ = train_batch_.merge(sensor_geometry, on = "sensor_id", how = "left")

In [14]:
train_batch_.x -= train_batch_.x.mean()
train_batch_.y -= train_batch_.y.mean()
train_batch_.z -= train_batch_.z.mean()

train_batch_ = train_batch_.merge(train_meta_, on = "event_id", how = "left")

train_batch_.head(10)

,event_id,sensor_id,time,charge,auxiliary,x,y,z,batch_id,first_pulse_index,last_pulse_index,azimuth,zenith,auxiliary_ratio
0,404418940,578,5951,0.375,True,33.416087,-319.273333,-70.711594,125,9812188,9812256,4.984989,0.490384,0.956522
1,404418940,2884,6060,1.325,True,469.926087,173.456667,514.548406,125,9812188,9812256,4.984989,0.490384,0.956522
2,404418940,3519,6060,0.425,True,503.146087,303.436667,-79.211594,125,9812188,9812256,4.984989,0.490384,0.956522
3,404418940,63,6308,0.225,True,-134.923913,-455.893333,527.368406,125,9812188,9812256,4.984989,0.490384,0.956522
4,404418940,2968,6698,0.925,True,574.246087,216.476667,95.818406,125,9812188,9812256,4.984989,0.490384,0.956522
5,404418940,1929,6842,1.275,True,-326.513913,-47.873333,425.308406,125,9812188,9812256,4.984989,0.490384,0.956522
6,404418940,3802,6861,1.275,True,52.136087,338.526667,228.458406,125,9812188,9812256,4.984989,0.490384,0.956522
7,404418940,1281,7035,0.725,True,-494.553913,-184.603333,218.528406,125,9812188,9812256,4.984989,0.490384,0.956522
8,404418940,2122,7424,2.575,True,44.166087,10.676667,204.878406,125,9812188,9812256,4.984989,0.490384,0.956522
9,404418940,938,7821,1.775,True,-168.523913,-242.233333,-70.771594,125,9812188,9812256,4.984989,0.490384,0.956522


In [15]:
zenith_true = train_batch_.zenith[0]
azimuth_true = train_batch_.azimuth[0]

vector = np.array([
    np.sin(zenith_true) * np.cos(azimuth_true),
    np.sin(zenith_true) * np.sin(azimuth_true),
    np.cos(azimuth_true)
])

vector_escalar = np.array([-500, 500])

x = vector[0] * vector_escalar 
y = vector[1] * vector_escalar 
z = vector[2] * vector_escalar 

direccion = pd.DataFrame({"x" : x, "y" : y, "z": z})
direccion.head()

,x,y,z
0,-63.400333,226.787019,-134.617995
1,63.400333,-226.787019,134.617995


## <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">PCA vs Noisy events</p> 

In [16]:
pca = PCA(n_components = 1).fit(train_batch_[train_batch_["charge"] > train_batch_["charge"].mean()][["x", "y", "z"]])

In [17]:
vector_pca = pca.components_[0]

x_pca = vector_pca[0] * vector_escalar
y_pca = vector_pca[1] * vector_escalar
z_pca = vector_pca[2] * vector_escalar

vector_pca_df = pd.DataFrame({'x_pca': x_pca, 'y_pca': y_pca, 'z_pca': z_pca})

In [18]:
for i, component in enumerate(pca.components_):
    print(f"component {i + 1}: {pca.explained_variance_ratio_[i]:.2%}")

component 1: 60.98%


In [19]:
zenith = np.arccos(vector_pca[2])
azimuth = np.arctan2(vector_pca[1], vector_pca[0])
if azimuth < 0:
    azimuth = 2*np.pi + azimuth

event_error = angular_dist_score(azimuth_true, zenith_true, azimuth, zenith)
print(f'The value of error: {event_error:.2f}')

The value of error: 0.61


In [20]:
auxiliary = px.scatter_3d(train_batch_[train_batch_.auxiliary], x = "x", y = "y", z = "z", size = "charge")
no_auxiliary = px.scatter_3d(train_batch_[~train_batch_.auxiliary], x = "x", y = "y", z = "z", size = "charge", color_discrete_sequence = ["aquamarine"])

true_direccion = px.line_3d(direccion, x = "x", y = "y", z = "z", color_discrete_sequence = ["white"])

pca_direccion = px.line_3d(vector_pca_df, x = "x_pca", y = "y_pca", z = "z_pca", color_discrete_sequence = ["steelblue"])

fig = go.Figure(data = auxiliary.data + no_auxiliary.data + true_direccion.data + pca_direccion.data)
fig.update_layout(template='plotly_dark')

fig.show()

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Non Auxiliar events </p>

In [21]:
example = train_meta_[train_meta_.auxiliary_ratio < .5]
id_example = example["event_id"][:1].values[0]


In [22]:
train_batch = pq.read_pandas("/kaggle/input/icecube-neutrinos-in-deep-ice/train/batch_125.parquet").to_pandas().reset_index()

In [23]:
train_batch_1 = train_batch[train_batch.event_id == id_example]

In [24]:
train_batch_1 = train_batch_1.merge(sensor_geometry, on = "sensor_id", how = "left")

In [25]:
train_batch_1.x -= train_batch_1.x.mean()
train_batch_1.y -= train_batch_1.y.mean()
train_batch_1.z -= train_batch_1.z.mean()

train_batch_1.x.describe()

count    1.070000e+02
mean    -1.580460e-14
std      1.545941e+02
min     -5.560850e+02
25%      3.063495e+01
50%      3.132495e+01
75%      4.676495e+01
max      4.126650e+02
Name: x, dtype: float64

In [26]:
event_pulses = train_batch_1.merge(example, on = "event_id", how = "left")

In [27]:
zenith_true = event_pulses.zenith[0]
azimuth_true = event_pulses.azimuth[0]

vector = np.array([
    np.sin(zenith_true) * np.cos(azimuth_true),
    np.sin(zenith_true) * np.sin(azimuth_true),
    np.cos(azimuth_true)
])

vector_escalar = np.array([-500, 500])

x = vector[0] * vector_escalar 
y = vector[1] * vector_escalar 
z = vector[2] * vector_escalar 

direccion = pd.DataFrame({"x" : x, "y" : y, "z": z})
direccion.head()

,x,y,z
0,-437.506842,241.016767,-437.943849
1,437.506842,-241.016767,437.943849


In [28]:
pca = PCA(n_components = 1).fit(event_pulses[~event_pulses["auxiliary"]][["x", "y", "z"]])

In [29]:
pca_vector = pca.components_[0]

# Calculating the first principal component vector
vector_pca = pca.components_[0]
vector_base = np.array([-500, 500])
x_pca = vector_base*vector_pca[0]
y_pca = vector_base*vector_pca[1]
z_pca = vector_base*vector_pca[2]
vector_pca_df = pd.DataFrame({'x_pca': x_pca, 'y_pca': y_pca, 'z_pca': z_pca})

In [30]:
for i, component in enumerate(pca.components_):
    print("{} component: {}% of initial variance".format(i + 1, 
          round(100 * pca.explained_variance_ratio_[i], 2)))

1 component: 98.91% of initial variance


In [31]:
zenith = np.arccos(vector_pca[2])
azimuth = np.arctan2(vector_pca[1], vector_pca[0])
if azimuth < 0:
    azimuth = 2*np.pi + azimuth

event_error = angular_dist_score(azimuth_true, zenith_true, azimuth, zenith)
print(f'The value of error: {event_error:.2f}')

The value of error: 1.06


In [32]:
# Plotting auxiliary pulses
fig_auxiliary = px.scatter_3d(event_pulses.loc[event_pulses.auxiliary],
                              x='x', y='y', z='z', opacity=0.5, color_discrete_sequence=['blue'])
# Plotting non-auxiliary pulses
fig_non_auxiliary = px.scatter_3d(event_pulses.loc[~event_pulses.auxiliary],
                              x='x', y='y', z='z', opacity=0.5, color_discrete_sequence=['aquamarine'])
# Plotting the true vector
fig_line = px.line_3d(direccion, x="x", y="y", z="z", color_discrete_sequence=['white'])

# Plotting the first principal component vector
fig_pca_line = px.line_3d(vector_pca_df, x="x_pca", y="y_pca", z="z_pca", color_discrete_sequence=['steelblue'])

fig_auxiliary.update_traces(marker_size=event_pulses.charge*10)
fig_non_auxiliary.update_traces(marker_size=event_pulses.charge*10)

fig = go.Figure(data = fig_auxiliary.data + fig_non_auxiliary.data + fig_line.data + fig_pca_line.data)
fig.update_layout(template='plotly_dark')
fig.show()

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Ranked Pulses</p>

In [33]:
train_meta_sample = train_meta_[:10000].copy()
train_meta_sample.head()

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith,auxiliary_ratio
0,125,403533054,0,64,5.036863,1.047974,0.692308
1,125,403533131,65,129,5.888834,0.806877,0.523077
2,125,403533142,130,236,5.779661,1.615474,0.317757
3,125,403533173,237,286,0.193563,1.638576,0.740000
4,125,403533184,287,319,2.306770,2.450495,0.787879


In [34]:
train_sample = train_batch.loc[: train_meta_sample.iloc[-1].last_pulse_index.astype(int)]

In [35]:
train_sample = train_sample.merge(sensor_geometry, on = "sensor_id", how = "left")

train_sample['x'] = train_sample['x'] - train_sample.groupby('event_id').x.transform('mean')
train_sample['y'] = train_sample['y'] - train_sample.groupby('event_id').y.transform('mean')
train_sample['z'] = train_sample['z'] - train_sample.groupby('event_id').z.transform('mean')

print(len(train_sample))

train_sample.head()

1817022


,event_id,sensor_id,time,charge,auxiliary,x,y,z
0,403533054,1166,6055,1.425,True,487.007692,-168.362,187.990462
1,403533054,2932,6279,2.725,True,632.207692,168.608,-248.699538
2,403533054,4165,6392,1.975,True,14.707692,414.948,208.670462
3,403533054,486,6415,0.775,True,72.107692,-343.592,529.990462
4,403533054,4165,6429,0.725,True,14.707692,414.948,208.670462


In [36]:
#the speed of light
c_const = 0.299792458

x_min, x_max = sensor_geometry.x.agg([min, max]).values
y_min, y_max = sensor_geometry.y.agg([min, max]).values
z_min, z_max = sensor_geometry.z.agg([min, max]).values

detector_length = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2 + (z_max - z_min)**2)
time_valid_length = detector_length / c_const
print(f"time_valid_length : {time_valid_length:.2f} m/ns")

time_valid_length : 6199.70 m/ns


In [37]:
def get_ranked_pulses(event_pulses):
    event_pulses["time_from_start"] = event_pulses.time.values - event_pulses.time.min()
    time_peak_charge = event_pulses.loc[event_pulses['charge'].argmax(), 'time_from_start']
    time_valid_min = time_peak_charge - time_valid_length
    time_valid_max = time_peak_charge + time_valid_length
    time_valid_window = ((event_pulses['time_from_start'] > time_valid_min) * 
                  (event_pulses['time_from_start'] < time_valid_max))
    event_pulses['pulse_rank'] = 2 * (1 - event_pulses['auxiliary'].values) + (time_valid_window)
    ranked_pulses = event_pulses.drop('time_from_start', axis=1)
    return ranked_pulses

In [38]:
def get_angles(event_pulses):
    event_pulses = (get_ranked_pulses(event_pulses)).query('pulse_rank == 3')
    try:
        pca = PCA(n_components=1).fit(event_pulses[['x', 'y', 'z']])
        vector = pca.components_[0]

        # zero-centering of the coordinates
        event_pulses.x = event_pulses.x - event_pulses.x.mean()
        event_pulses.y = event_pulses.y - event_pulses.y.mean()
        event_pulses.z = event_pulses.z - event_pulses.z.mean()

        # calculating a distance of a point from the first principal component, that goes through the origin
        event_pulses['distance'] = (event_pulses.x * vector[0] + event_pulses.y * vector[1] +
                                              event_pulses.z * vector[2]) / np.linalg.norm(vector)

        # Flip the vector direction if it points away from the neutrino origin
        if (event_pulses.loc[event_pulses.distance > 0].time.mean() >
            event_pulses.loc[event_pulses.distance < 0].time.mean()):
            vector = -1 * vector
            
        vector = np.clip(vector, -1, 1)
        
        zenith = np.arccos(vector[2])
        azimuth = np.arctan2(vector[1], vector[0])
        if azimuth < 0:
            azimuth = 2 * np.pi + azimuth

    except:
        zenith, azimuth = 0., 0.
        
    return zenith, azimuth

In [39]:
%%time

train_meta_sample[['azimuth_pred', 'zenith_pred']] = None
for i in range(len(train_meta_sample)):
    row = train_meta_sample.iloc[i]
    event_pulses = train_sample.iloc[ \
        row.first_pulse_index.astype(int):row.last_pulse_index.astype(int)+1].reset_index().copy()
    zenith, azimuth = get_angles(event_pulses)
    train_meta_sample.loc[i, 'zenith_pred'] = zenith
    train_meta_sample.loc[i, 'azimuth_pred'] = azimuth

CPU times: user 3min 2s, sys: 41.6 s, total: 3min 44s
Wall time: 2min 36s


In [40]:
sample_error = angular_dist_score(train_meta_sample.azimuth.to_list(), train_meta_sample.zenith.to_list(),
                   train_meta_sample.azimuth_pred.astype(float).to_list(),
                   train_meta_sample.zenith_pred.astype(float).to_list())

print(f'The value of error: {sample_error:.3f}')

The value of error: 1.208


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">Sample Submission</p>

In [41]:
test_meta = pd.read_parquet("/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet")
test_meta.head()

,batch_id,event_id,first_pulse_index,last_pulse_index
0,661,2092,0,298
1,661,7344,299,334
2,661,9482,335,377


In [42]:
def load_test_batch(batch_id):
    test_batch = pd.read_parquet("/kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_661.parquet") \
                                .reset_index()
    test_batch[['x', 'y', 'z']] = sensor_geometry.loc[test_batch.sensor_id].reset_index()[['x', 'y', 'z']]
    test_batch.x = test_batch.x - test_batch.groupby('event_id').x.transform('mean')
    test_batch.y = test_batch.y - test_batch.groupby('event_id').y.transform('mean')
    test_batch.z = test_batch.z - test_batch.groupby('event_id').z.transform('mean')
    return test_batch

In [43]:
for batch in test_meta.batch_id.unique():
    test_batch = load_test_batch(batch)
    test_meta_batch = test_meta.loc[test_meta.batch_id == batch]

    pulses_info = []
    for i in range(len(test_meta_batch)):
        row = test_meta_batch.iloc[i]
        pulses_info.append(test_batch.iloc[ \
            row.first_pulse_index.astype(int):row.last_pulse_index.astype(int)+1].reset_index().copy())
        
    results = []
    for result in map(get_angles, pulses_info):
        results.append(result)
            
    test_meta.loc[test_meta_batch.index, 'zenith'] = [results[i][0] for i in range(len(results))]
    test_meta.loc[test_meta_batch.index, 'azimuth'] = [results[i][1] for i in range(len(results))]
    
test_meta.head()

,batch_id,event_id,first_pulse_index,last_pulse_index,zenith,azimuth
0,661,2092,0,298,1.588634,5.351306
1,661,7344,299,334,3.141593,3.141593
2,661,9482,335,377,1.591759,4.345112


In [44]:
submission = test_meta[['event_id', 'azimuth', 'zenith']]
submission.to_csv('submission.csv', index=False)
submission.head()

,event_id,azimuth,zenith
0,2092,5.351306,1.588634
1,7344,3.141593,3.141593
2,9482,4.345112,1.591759
